# Wind Farm Spotter: Inference Test (Batch, Single) [Swift, GPU Runtime]

### **Environment Setup**
First setting up the colab/jupyter notebook with necessary Swift packages.

In [0]:
%install '.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")' Path
%install '.package(url: "https://github.com/JohnSundell/ShellOut.git", from: "2.0.0")' ShellOut
%install '.package(url: "https://github.com/IBM-Swift/SwiftyRequest.git", from: "2.0.6")' SwiftyRequest
%install '.package(url: "https://github.com/JustHTTP/Just", from: "0.7.1")' Just
%install '.package(url: "https://github.com/JohnSundell/Files.git", from: "3.1.0")' Files
%install '.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")' NotebookExport
%install '.package(url: "https://github.com/weichsel/ZIPFoundation/", .upToNextMajor(from: "0.9.0"))' ZIPFoundation
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

**Swift Imports:**

In [0]:
import Foundation
import Python
import SwiftyRequest
import Path
import Files
import ShellOut
import FoundationNetworking
import ZIPFoundation

**Python Imports:**

We'll be using the shellOut package to run a shell command to **Install pytorchcv**.

In [3]:
try shellOut(to: "pip install pytorchcv")

"Requirement already satisfied: pytorchcv in /usr/local/lib/python3.6/dist-packages (0.0.52)\nRequirement already satisfied: numpy in /usr/local/lib/python3.6/dist-packages (from pytorchcv) (1.17.4)\nRequirement already satisfied: requests in /usr/local/lib/python3.6/dist-packages (from pytorchcv) (2.21.0)\nRequirement already satisfied: idna<2.9,>=2.5 in /usr/local/lib/python3.6/dist-packages (from requests->pytorchcv) (2.8)\nRequirement already satisfied: chardet<3.1.0,>=3.0.2 in /usr/local/lib/python3.6/dist-packages (from requests->pytorchcv) (3.0.4)\nRequirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.6/dist-packages (from requests->pytorchcv) (2019.9.11)\nRequirement already satisfied: urllib3<1.25,>=1.21.1 in /usr/local/lib/python3.6/dist-packages (from requests->pytorchcv) (1.24.3)"


In [4]:
try shellOut(to: "pip3 install nvidia-ml-py3")

"Requirement already satisfied: nvidia-ml-py3 in /usr/local/lib/python3.6/dist-packages (7.352.0)"


In [0]:
let warnings = Python.import("warnings")
warnings.filterwarnings("ignore")
let os = Python.import("os")
let pd = Python.import("pandas")
let fastai = Python.import("fastai")
let fastai_vision = Python.import("fastai.vision") 
let torch = Python.import("torch")
let torchvision = Python.import("torchvision")
let plt = Python.import("matplotlib.pyplot")
let py_time = Python.import("time")
let sys = Python.import("sys")
let pytorchcv = Python.import("pytorchcv")
let model_provider = Python.import("pytorchcv.model_provider")
let gc = Python.import("gc")


In [6]:
print("Fastai Version: ", fastai.__version__)

Fastai Version:  1.0.59


**CUDA Check:**

In [0]:
fastai.torch_core.defaults.device = torch.device("cuda")

In [8]:
torch.cuda.is_available()

True


In [9]:
torch.cuda.current_device()

0


In [10]:
torch.cuda.device_count()

1


In [11]:
torch.cuda.get_device_name(0)

Tesla P100-PCIE-16GB


### **Accessing the Image Data**

In [12]:
print(Folder.current)

Folder(name: content, path: /content/)


In [0]:
let fileManager = FileManager()

In [0]:
let test_data_zip = Path("/content/test_single.zip")!
fileManager.unzipItem(at: test_data_zip.url, to: Path("/content/")!.url)

In [0]:
let test_data_labeled_zip = Path("/content/test.zip")!
fileManager.unzipItem(at: test_data_labeled_zip.url, to: Path("/content/")!.url)

### **Test Set Data Loaders**
Next we'll create Data Loaders for both EfficientNet-b1 and Inception-v3. 

In [0]:
let bs = 32 
let size1 = 320
let size2 = 363
let tfm_train:PythonObject = [fastai_vision.transform.crop_pad()]
let tfm_valid:PythonObject = []
let tfms = Python.tuple([tfm_train, tfm_valid])

Since we want to be able to tabulate the accuracy of our predictions, we need to use a labeled test set. The directory entitled "test" is a labeled set, while the directory entitled "test_single" is unlabeled.

In [0]:
let ImageList = fastai_vision.ImageList
//let ptcv_get_model = model_provider.get_model
let il = ImageList.from_folder("/content/test")
let ils = il.split_none()
let ll = ils.label_from_folder()
ll.valid = ll.train
ll.test = ll.train

**Inference Data Loader for EfficientNet:**

In [18]:
let effnet_ll = ll 
effnet_ll.transform.dynamicallyCall(withKeywordArguments: ["tfms": tfms as PythonConvertible, "size": size1 as PythonConvertible])
let effnet_data = effnet_ll.databunch.dynamicallyCall(withKeywordArguments: ["bs": bs])
effnet_data.normalize(fastai_vision.data.imagenet_stats)

ImageDataBunch;

Train: LabelList (49 items)
x: ImageList
Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320)
y: CategoryList
no_turbines_low_potential,no_turbines_low_potential,turbines_medium_capacity,turbines_medium_capacity,turbines_medium_capacity
Path: /content/test;

Valid: LabelList (49 items)
x: ImageList
Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320)
y: CategoryList
no_turbines_low_potential,no_turbines_low_potential,turbines_medium_capacity,turbines_medium_capacity,turbines_medium_capacity
Path: /content/test;

Test: LabelList (49 items)
x: ImageList
Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320)
y: CategoryList
no_turbines_low_potential,no_turbines_low_potential,turbines_medium_capacity,turbines_medium_capacity,turbines_medium_capacity
Path: /content/test


**Inference Data Loader for Inception:**

In [19]:
let incept_ll = ll 
incept_ll.transform.dynamicallyCall(withKeywordArguments: ["tfms": tfms as PythonConvertible, "size": size2 as PythonConvertible])
let incept_data = incept_ll.databunch.dynamicallyCall(withKeywordArguments: ["bs": bs])
incept_data.normalize(fastai_vision.data.imagenet_stats)

ImageDataBunch;

Train: LabelList (49 items)
x: ImageList
Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363)
y: CategoryList
no_turbines_low_potential,no_turbines_low_potential,turbines_medium_capacity,turbines_medium_capacity,turbines_medium_capacity
Path: /content/test;

Valid: LabelList (49 items)
x: ImageList
Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363)
y: CategoryList
no_turbines_low_potential,no_turbines_low_potential,turbines_medium_capacity,turbines_medium_capacity,turbines_medium_capacity
Path: /content/test;

Test: LabelList (49 items)
x: ImageList
Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363)
y: CategoryList
no_turbines_low_potential,no_turbines_low_potential,turbines_medium_capacity,turbines_medium_capacity,turbines_medium_capacity
Path: /content/test


### **Reloading the Fast.ai Learner**
Fast.ai provides a useful method named **load_learner()** to reinstantiate an instance model from a pickle file. The loaded learner recalls all the configurations(e.g., transformations, batch size, etc.) used during training. load_learner() also gives us the option of assigning a list of images (unlabeled) right from the start. To run our Single Image Inference Test, we'll use the single_test directory.

In [0]:
let inference_test_batch_efficientnet = fastai_vision.load_learner.dynamicallyCall(withKeywordArguments: ["path": "/content/", "file":"effnet-b1-c98.pkl"])
let inference_test_batch_inceptionv3 = fastai_vision.load_learner.dynamicallyCall(withKeywordArguments: ["path": "/content/", "file":"incept-v3-95-2.pkl"])

In [0]:
inference_test_batch_efficientnet.data = effnet_data
inference_test_batch_inceptionv3.data = incept_data

### **Batch Inference**

In [22]:
gc.collect()

0


**EfficientNet Batch Inference:**

In [0]:
inference_test_batch_efficientnet.model.cuda()
inference_test_batch_efficientnet.model.eval()

In [25]:
let eff_b_start = py_time.time()
let effnet_b_results = inference_test_batch_efficientnet.get_preds.dynamicallyCall(withKeywordArguments: ["ds_type": fastai.basic_data.DatasetType.Test])
let eff_b_end = py_time.time()
let eff_b_elapsed = eff_b_end - eff_b_start
print("Swift Batch Elapsed Time (EfficientNet): \(eff_b_elapsed)")

Swift Batch Elapsed Time (EfficientNet): 10.292629718780518


**Performance**:

In [26]:
let effnet_top_1 = fastai.metrics.top_k_accuracy(effnet_b_results[0], effnet_b_results[1], 1)
let effnet_top_3 = fastai.metrics.top_k_accuracy(effnet_b_results[0], effnet_b_results[1], 3)
let effnet_top_5 = fastai.metrics.top_k_accuracy(effnet_b_results[0], effnet_b_results[1], 5)

print("Top 1% Accuracy: ", effnet_top_1)
print("Top 3% Accuracy: ", effnet_top_3)
print("Top 5% Accuracy: ", effnet_top_5)

Top 1% Accuracy:  tensor(0.7347)
Top 3% Accuracy:  tensor(0.9184)
Top 5% Accuracy:  tensor(0.9796)


**Results:**

In [97]:
//inference_test_efficientnet.show_results()

None


**Inceptionv3 Batch Inference:**

In [27]:
gc.collect()

0


In [0]:
inference_test_batch_inceptionv3.model.cuda()
inference_test_batch_inceptionv3.model.eval()

In [29]:
let incept_b_start = py_time.time()
let incept_b_results = inference_test_batch_inceptionv3.get_preds.dynamicallyCall(withKeywordArguments: ["ds_type": fastai.basic_data.DatasetType.Test])
let incept_b_end = py_time.time()
let incept_b_elapsed = incept_b_end - incept_b_start
print("Swift Batch Elapsed Time (Inceptionv3): \(incept_b_elapsed)")

Swift Batch Elapsed Time (Inceptionv3): 10.318971157073975


**Performance**:

In [30]:
let incept_top_1 = fastai.metrics.top_k_accuracy(incept_b_results[0], incept_b_results[1], 1)
let incept_top_3 = fastai.metrics.top_k_accuracy(incept_b_results[0], incept_b_results[1], 3)
let incept_top_5 = fastai.metrics.top_k_accuracy(incept_b_results[0], incept_b_results[1], 5)

print("Top 1% Accuracy: ", incept_top_1)
print("Top 3% Accuracy: ", incept_top_3)
print("Top 5% Accuracy: ", incept_top_5)

Top 1% Accuracy:  tensor(0.6939)
Top 3% Accuracy:  tensor(0.9592)
Top 5% Accuracy:  tensor(0.9796)


**Results:**

In [119]:
//inference_test_inceptionv3.show_results.dynamicallyCall(withKeywordArguments: ["rows":9])

None


### **Inference Testing (Single)**
Fast.ai's **load_learner()** method also gives us the option of assigning a list of images (unlabeled) right from the start. To run our Single Image Inference Test, we'll use the single_test directory to create a Fast.ai **ImageList**.

In [0]:
let inference_test_single_efficientnet = fastai_vision.load_learner.dynamicallyCall(withKeywordArguments: ["path": "/content/", "file":"effnet-b1-c98.pkl", "test": ImageList.from_folder("/content/test_single")])
let inference_test_single_inceptionv3 = fastai_vision.load_learner.dynamicallyCall(withKeywordArguments: ["path": "/content/", "file":"incept-v3-95-2.pkl", "test": ImageList.from_folder("/content/test_single")])

**Create a name list:**

In [0]:
let test_files = Path("/content/test_single")!.url.path
let test_file_list = os.listdir(test_files)
let df = pd.DataFrame(test_file_list)
//df

**EfficientNet Single Inference:**

In [33]:
gc.collect()

7


Here, we set our model to evaluation mode...

In [0]:
inference_test_single_efficientnet.model.cuda()
inference_test_single_efficientnet.model.eval()

and loop through the learner's test dataset, making a prediction for each item, and storing the result in our eff_preds dictionary: 

In [35]:
var eff_preds:[String:String] = [:]

let eff_start = py_time.time()
for img in 0...48 {
  var name = String(test_file_list[img])!
  var p = inference_test_single_efficientnet.predict(inference_test_single_efficientnet.data.test_dl.dl.dataset.x[img]).tuple3
  eff_preds[name] = String(Python.str(p.0))!
}
let eff_stop = py_time.time()
let eff_elapsed = eff_stop - eff_start
print("Swift Elapsed Time: \(eff_elapsed)")


Swift Elapsed Time: 12.2718825340271


In [0]:
let eff_pred_list:PythonObject = PythonObject(eff_preds).items()
//print(pred_list)

**Results:**

In [37]:
let eff_results = pd.DataFrame.dynamicallyCall(withKeywordArguments: ["data": Array(eff_pred_list), "columns": ["file_name", "id"]])
print(eff_results)

                                file_name                          id
0   test_no_turbines_high_potential_8.jpg  no_turbines_high_potential
1                 test_low_capacity_7.jpg       turbines_low_capacity
2             test_medium_capacity_11.jpg   no_turbines_low_potential
3   test_no_turbines_high_potential_1.jpg  no_turbines_high_potential
4              test_medium_capacity_1.jpg    turbines_medium_capacity
5     test_no_turbines_no_potential_3.jpg    no_turbines_no_potential
6             test_medium_capacity_12.jpg    turbines_medium_capacity
7    test_no_turbines_low_potential_1.jpg   no_turbines_low_potential
8   test_no_turbines_high_potential_5.jpg  no_turbines_high_potential
9    test_no_turbines_med_potential_1.jpg   no_turbines_med_potential
10                test_low_capacity_3.jpg       turbines_low_capacity
11                test_low_capacity_8.jpg       turbines_low_capacity
12             test_medium_capacity_7.jpg   no_turbines_med_potential
13                te

**Inception-v3 Single Inference:**

In [38]:
gc.collect()

25


We'll do the same for Inception-v3:

In [0]:
inference_test_single_inceptionv3.model.cuda()
inference_test_single_inceptionv3.model.eval()

In [40]:
var incept_preds:[String:String] = [:]

let incept_start = py_time.time()
for img in 0...48 {
  var name = String(test_file_list[img])!
  var p = inference_test_single_inceptionv3.predict(inference_test_single_inceptionv3.data.test_dl.dl.dataset.x[img]).tuple3
  incept_preds[name] = String(Python.str(p.0))!
}
let incept_stop = py_time.time()
let incept_elapsed = incept_stop - incept_start 
print("Elapsed Time: \(incept_elapsed)")

Elapsed Time: 12.88382887840271


In [0]:
let incept_pred_list:PythonObject = PythonObject(incept_preds).items()
//print(pred_list)

**Results:**

In [42]:
let incept_results = pd.DataFrame.dynamicallyCall(withKeywordArguments: ["data": Array(incept_pred_list), "columns": ["file_name", "id"]])
print(incept_results)

                                file_name                          id
0                test_high_capacity_2.jpg    turbines_medium_capacity
1                 test_low_capacity_6.jpg       turbines_low_capacity
2                 test_low_capacity_9.jpg   no_turbines_low_potential
3   test_no_turbines_high_potential_1.jpg  no_turbines_high_potential
4    test_no_turbines_low_potential_2.jpg   no_turbines_low_potential
5     test_no_turbines_no_potential_4.jpg    no_turbines_no_potential
6                test_high_capacity_8.jpg      turbines_high_capacity
7                 test_low_capacity_7.jpg       turbines_low_capacity
8             test_medium_capacity_13.jpg    turbines_medium_capacity
9     test_no_turbines_no_potential_2.jpg    no_turbines_no_potential
10                test_low_capacity_4.jpg    turbines_medium_capacity
11                test_low_capacity_5.jpg   no_turbines_low_potential
12             test_medium_capacity_1.jpg    turbines_medium_capacity
13  test_no_turbines